In [ ]:
! rm -rf *
! git clone https://github.com/bdsp-core/IIIC-SPaRCNet SPaRCNet

In [ ]:
from glob import glob
all_mat_files = glob("/kaggle/working/SPaRCNet/code_for_figures/Data/**/*.mat", recursive=True)

In [ ]:
import scipy.io
extra_dataset_mat_files = []
for mat_file in all_mat_files:
    mat = scipy.io.loadmat(mat_file)
    if 'seg' in mat:
        print(mat_file, len(mat), mat.keys())
        extra_dataset_mat_files.append(mat_file)
    else:
        print('skip', mat_file, len(mat), mat.keys())

In [ ]:
! mkdir /kaggle/working/dataset
! mkdir /kaggle/working/dataset/train_eegs
! mkdir /kaggle/working/model

In [ ]:
import pandas as pd
train = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
columns = list(train.columns)
targets = [x for x in columns if '_vote' in x]
str(columns), str(targets)

In [ ]:
extra_train = []
import scipy.io
for seq_id, mat_file in enumerate(extra_dataset_mat_files):
    details = {x:'' for x in columns}
    label = mat_file.split("/")[-1].split("_")[0]
    if label == "SZ":
        label = "Seizure"
    mat = scipy.io.loadmat(mat_file)
    print(seq_id, label, mat['seg'].shape)
    details['eeg_id'] = f"extra_{seq_id}"
    details['eeg_sub_id'] = 0
    details['eeg_label_offset_seconds'] = 0
    details['spectrogram_id'] = "-"
    details['spectrogram_sub_id'] = "-"
    details['spectrogram_label_offset_seconds'] = "-"
    details['label_id'] = details['eeg_id']
    details['patient_id'] = "-"
    for x in targets:
        details[x] = 0
    details[f"{label.lower()}_vote"] = 10 # (0+20/2)
    extra_train.append(details)
    
    mat_df = pd.DataFrame(mat['seg'])
    mat_df.to_parquet(f"/kaggle/working/dataset/train_eegs/{details['eeg_id']}.parquet")

In [ ]:
train_extra = pd.DataFrame(extra_train)
train_extra

In [ ]:
train_extra.to_csv("/kaggle/working/dataset/train_extra.csv")

In [ ]:
! cp /kaggle/working/SPaRCNet/SPaRCNet/model_1130.pt /kaggle/working/model/SPaRCNet.pt